In [2]:
# cell pose is machine learning model for segmenting cells
#from cellpose import models, utils, io

# for clearing jpynb output
from IPython.display import clear_output

# import necessary libraries
from scipy.ndimage import binary_fill_holes
#from mpl_toolkits.mplot3d import Axes3D

import math
import skimage
import numpy as np
from czifile import CziFile
from skimage import filters, morphology, segmentation
import numpy as np
from skimage.measure import regionprops
import os
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
from skimage.measure import label, regionprops
from skimage.io import imread
from skimage.measure import label
import czifile as czi
import pandas as pd
from skimage.filters import gaussian, threshold_otsu, sobel, median
from skimage.filters import threshold_otsu
from skimage import exposure
from skimage import morphology
from skimage.measure import label
from skimage.morphology import remove_small_objects
from skimage.morphology import binary_dilation, disk
from os import listdir
from os.path import isfile, join

In [2]:
def mitochondrial_mass_analysis(images_to_analyze, folder_path):
    '''
    mitochondrial_mass_analysis()
    Splits, adjusts, thresholds, and measures the area of the mask of each channel over each slice of a Z stack
    '''
    removed_files = []

    def mask_calcein(calcein_unp):
        '''
        mask_calcein()
        thresholds and masks the green channel
        '''
        # if low contrast match the cumu distribution to a more highly contrasted image
        low_contrast_calcein = skimage.exposure.is_low_contrast(calcein_unp, lower_percentile=25, upper_percentile=75)
        # local equalize histogram works much better for the lower contrast calcein slices and doesn't hurt the higher contrast slices
        equalize_calcein = skimage.exposure.equalize_adapthist(calcein_unp)

        calcein_li_threshold = skimage.filters.threshold_li(equalize_calcein,
                                                        initial_guess=np.mean(equalize_calcein))
        binary_image_calcein = equalize_calcein > calcein_li_threshold

        if low_contrast_calcein:
            #print('this slice is calcein low contrast')
            mask_size1 = np.count_nonzero(equalize_calcein > calcein_li_threshold)
            binary_image_calcein = np.logical_or(equalize_calcein > calcein_li_threshold, equalize_calcein > np.percentile(equalize_calcein, 15))
            mask_size2 = np.count_nonzero(np.logical_or(equalize_calcein > calcein_li_threshold, equalize_calcein > np.percentile(equalize_calcein, 15)))

            # if the difference between the two masks is too larger (over double original) don't use it
            if mask_size1 * 2 < mask_size2:
                binary_image_calcein = equalize_calcein > calcein_li_threshold
            
            calcein_binary_cleaned = skimage.morphology.remove_small_holes(binary_image_calcein, 50)
        
        # fill anything that is above the twenty fifth percentile
        calcein_binary_cleaned = skimage.morphology.remove_small_holes(binary_image_calcein, 20)
        calcein_binary_cleaned = skimage.morphology.remove_small_objects(calcein_binary_cleaned, min_size=75)
        # find the edges of the image - will try to use this to make a polygon
        calcein_edges = skimage.feature.canny(calcein_binary_cleaned)
        # return three images
        return (calcein_unp, calcein_binary_cleaned, calcein_edges, calcein_li_threshold)

    def mask_hoechst(hoechst_unp):
        '''
        mask_hoechst()
        thresholds and masks the blue channel
        '''
        # if low contrast match the cumu distribution to a more highly contrasted image
        low_contrast_hoechst = skimage.exposure.is_low_contrast(hoechst_unp, lower_percentile=25, upper_percentile=75)
        # local equalize histogram works much better for the lower contrast calcein slices and doesn't hurt the higher contrast slices
        equalize_hoechst = skimage.exposure.equalize_adapthist(hoechst_unp)

        hoechst_li_threshold = skimage.filters.threshold_li(equalize_hoechst,
                                                        initial_guess=np.mean(equalize_hoechst))
        binary_image_hoechst = equalize_hoechst > hoechst_li_threshold

        if low_contrast_hoechst:
            hoechst_binary_cleaned = skimage.morphology.remove_small_holes(binary_image_hoechst, 50)
        else: 
            hoechst_binary_cleaned = skimage.morphology.remove_small_holes(binary_image_hoechst, 20)

        hoechst_binary_cleaned = skimage.morphology.remove_small_objects(hoechst_binary_cleaned, min_size=75)
        # remove small dots and then fill in the nuclei
        hoechst_binary_cleaned = skimage.morphology.remove_small_holes(hoechst_binary_cleaned, area_threshold=35)
        hoechst_binary_filled = binary_fill_holes(hoechst_binary_cleaned)
        # find the edges of the image - will try to use this to make a polygon
        hoechst_edges = skimage.feature.canny(hoechst_binary_filled)

        return (hoechst_unp, hoechst_binary_filled, hoechst_edges, hoechst_li_threshold)

    def mask_tmrm(tmrm_unp):
        '''
        mask_tmrm()
        thresholds and masks the red channel
        '''
        # local equalize histogram works much better for the lower contrast calcein slices and doesn't hurt the higher contrast slices
        equalize_tmrm = skimage.exposure.equalize_adapthist(tmrm_unp)
        #if low_contrast_tmrm:
        tmrm_threshold = np.mean(equalize_tmrm) + (np.std(equalize_tmrm) * 1.85) #1.85 works well

        binary_image_tmrm = equalize_tmrm > tmrm_threshold
        
        tmrm_binary_cleaned = skimage.morphology.remove_small_holes(binary_image_tmrm, 20)
        tmrm_binary_cleaned = skimage.morphology.remove_small_objects(tmrm_binary_cleaned, min_size=30)
        
        tmrm_edges = skimage.feature.canny(tmrm_binary_cleaned)
        return (tmrm_unp, tmrm_binary_cleaned, tmrm_edges, tmrm_threshold)

    # new - the volumes will be reported as a data frame
    volume_rows = []

    # iterate over each z stack file
    for path in images_to_analyze:
        print(path)

        # get path of image (this just ensures you are looking in the necessary folder since the images_to_analyze has just the file names)
        image_path = folder_path + "\\" + path
        # read in the file
        czif = czi.CziFile(image_path)

        # grab the number of channels - if less than three exclude hoechst (Hoechst) channel analysis
        channel_number = czif.shape[(czif.axes).index("C")]
        hoechst_in = False
        if channel_number > 2:
            hoechst_in = True
            #print('hoechst in')
    
        # read in the file as an array
        image = czi.imread(image_path)
        # squeeze the image - remove empty arrays
        image_squeezed = np.squeeze(image)

        n_slices = image_squeezed.shape[1]
        # initialize lists for storing processed slices and the areas (number of pixels in the mask)
        calcein_masked_slices = []
        tmrm_masked_slices = []
        hoechst_masked_slices = []

        # for each stack, iterate over both channels - plot original, then plot the mask
        for current_slice in range(n_slices):
            unprocessed_slice = image_squeezed[:,current_slice,:,:]
            # grab the channel image from the larger czi file object
            tmrm_unp = unprocessed_slice[0,:,:]
            calcein_unp = unprocessed_slice[1,:,:]
            # comptue the threshold for each channel
            calcein_p = mask_calcein(calcein_unp)
            tmrm_p = mask_tmrm(tmrm_unp)
            calcein_masked_slices.append(calcein_p[1])
            tmrm_masked_slices.append(tmrm_p[1])
            # this can be removed if there is not confusion with how many channels each z stack has
            if hoechst_in == True:
                hoechst_unp = unprocessed_slice[2,:,:]
                hoechst_p = mask_hoechst(hoechst_unp)
                hoechst_masked_slices.append(hoechst_p[1])
                   
            
            if hoechst_in == True:
                fig, axes = plt.subplots(2, 3, figsize=(15, 9))
            else:
                fig, axes = plt.subplots(2, 2, figsize=(15, 9))
            # Plot each frame
            axes[0, 0].imshow(calcein_p[0])
            axes[0, 0].set_title(f'Calcein {current_slice + 1}')
            axes[0, 0].axis('off')

            axes[1, 0].imshow(calcein_p[1])
            axes[1, 0].set_title(f'Calcein mask {current_slice + 1}')
            axes[1, 0].axis('off')

            axes[0, 1].imshow(tmrm_p[0])
            axes[0, 1].set_title(f'TMRM {current_slice + 1}')
            axes[0, 1].axis('off')

            axes[1, 1].imshow(tmrm_p[1])
            axes[1, 1].set_title(f'TMRM mask {current_slice + 1}')
            axes[1, 1].axis('off')

            if hoechst_in == True:
                axes[0, 2].imshow(hoechst_p[0])
                axes[0, 2].set_title(f'hoechst {current_slice + 1}')
                axes[0, 2].axis('off')

                axes[1, 2].imshow(hoechst_p[1])
                axes[1, 2].set_title(f'hoechst mask {current_slice + 1}')
                axes[1, 2].axis('off')
            
            plt.show()

        # compute the volume of each channel by multiplying the number of non-zero values in the 3D matrix
        # would normally have to multiply by the distance on X,Y, and Z - but this will be cancelled out by taking a ratio
        # count the number of non-zero values (masked)
        if hoechst_in == True:
            volumes = [np.count_nonzero(np.dstack(arr_list)) for arr_list in [calcein_masked_slices, tmrm_masked_slices, hoechst_masked_slices, ]]
        else:
            volumes = [np.count_nonzero(np.dstack(arr_list)) for arr_list in [calcein_masked_slices, tmrm_masked_slices]]
            # add None value to fill for no hoechst staining
            volumes.append(None)
        
        # append the file name to the list and add that to the list of lists
        volumes.append(path)
        
        # Jiya's chunk for visually inspecting each image as it is printed
        add_df = input("Do you want to add this dataframe? (yes/no): ").lower()
        plt.close('all')
        plt.clf()
        clear_output(wait=True)
        if 'y' in add_df:
         #if basename in result_dict and i in result_dict[basename]:
            volume_rows.append(volumes)
        elif add_df == 'n':
            removed_files.append(path)
            with open(r"C:\Users\bs1250\Box\LAB\Lab Folder\WGCNA_Ben\Analysis\discarded_images_06192024.txt", 'w') as f:
                f.write(f"Discarded: {path}")
            f.close()
        elif add_df == 'stop':
            raise Exception('stop input')
    # convert into a pandas data frame
    
    results_df = pd.DataFrame(data=volume_rows, columns=['Calcein Volume', 'TMRM Volume', 'Hoechst Volume', 'File Name'])
    return results_df


In [ ]:
zstack_folder_path = "mitochondrial_mass_images"
zstack_files = [f for f in listdir(zstack_folder_path) if isfile(join(zstack_folder_path, f))]

exp_results = mitochondrial_mass_analysis(zstack_files, zstack_folder_path)

display(exp_results)